# Stage 7a — Transfer-IN Ranking Evaluation

## Hypothesis

For transfer-IN decisions, minimizing the risk of zero minutes matters.
Therefore:

```
score = p_play × mu_points
```

should outperform `mu_points` alone.

This contrasts with **captaincy** (Stage 6), where we found that availability weighting hurt performance. The reasoning is that transfer-IN decisions have higher downside risk: a captain who doesn't play might still have a vice-captain, but a transferred-in player who blanks wastes a valuable transfer.

## Policies Tested

| Policy | Score Formula | Rationale |
|--------|---------------|-----------|
| **Stage 7a** | `p_play × mu_points` | Availability-adjusted EV |
| Baseline A | `mu_points` | Pure upside |
| Baseline B | Random | Lower bound |
| Baseline C | `points_per_90_5` | Historical PPG |

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load evaluation results
eval_df = pd.read_csv("../storage/datasets/evaluation_transfer_in.csv")

print(f"Gameweeks evaluated: {eval_df['gw'].nunique()}")
print(f"Policies: {eval_df['policy_name'].unique().tolist()}")

## Summary Metrics

In [ ]:
# Compute metrics per policy
policies = [
    "stage_7a_p_play_x_mu_points",
    "baseline_a_mu_points",
    "baseline_b_random",
    "baseline_c_points_per_90",
]

results = []
for policy in policies:
    df = eval_df[eval_df["policy_name"] == policy]
    results.append({
        "Policy": policy,
        "Mean Regret": df["regret"].mean(),
        "Median Regret": df["regret"].median(),
        "% GW ≥ 10": f"{(df['regret'] >= 10).mean():.1%}",
        "Total Regret": df["regret"].sum(),
    })

summary = pd.DataFrame(results).sort_values("Mean Regret")
print(summary.to_string(index=False))

## Policy Comparison — Mean Regret

In [ ]:
# Bar chart comparing mean regret
fig, ax = plt.subplots(figsize=(10, 5))

policy_labels = ["p_play × mu_points", "mu_points", "random", "points_per_90"]
means = [eval_df[eval_df["policy_name"] == p]["regret"].mean() for p in policies]
colors = ["#e74c3c", "#2ecc71", "#95a5a6", "#9b59b6"]

bars = ax.bar(policy_labels, means, color=colors, edgecolor="black", alpha=0.8)

# Add value labels
for bar, val in zip(bars, means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
            f"{val:.2f}", ha="center", fontsize=11, fontweight="bold")

ax.set_ylabel("Mean Regret (pts/GW)")
ax.set_title("Transfer-IN Policy Comparison — Lower is Better")
ax.set_ylim(0, max(means) + 2)
ax.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

---

## Interpretation

### 1. Does availability-adjusted EV reduce transfer regret?

**No.** The `p_play × mu_points` policy has **higher** mean regret (6.91) than `mu_points` alone (6.23). The delta is +0.68 pts/GW — the same magnitude and direction as the captaincy result.

### 2. How does this differ from captaincy results?

**It doesn't.** The pattern is identical:

| Decision | p_play × mu_points | mu_points | Δ |
|----------|-------------------|-----------|---|
| Captain (Stage 6) | 6.91 | 6.23 | +0.68 |
| Transfer-IN (Stage 7a) | 6.91 | 6.23 | +0.68 |

This is not a coincidence — both decisions select from the same candidate pool using the same ranking method. The availability penalty hurts in both cases because the best players tend to have high `p_play` anyway (they're nailed-on starters).

### 3. Is the hypothesis supported or rejected?

**Rejected.** Availability-adjusted EV does NOT improve transfer-IN decisions.

The intuition that "transfer-IN should care about rotation" is appealing but empirically wrong. The top-ranked players by `mu_points` are typically guaranteed starters. Multiplying by `p_play ≈ 0.95` just dampens their scores without providing protective value.

### Conclusion

For single-GW transfer-IN ranking:

```python
def transfer_in_score(player):
    return player["mu_points"]  # Pure upside, same as captaincy
```

The `p_play` adjustment may be useful for **lower-ownership differential picks** or **multi-GW planning** where rotation risk accumulates, but not for top-end single-GW decisions.